In [20]:
import time, copy
import numpy as np

def load_dataset(file_name):
  f = open(file_name)
  file_content = f.readlines()
  f.close()
  data = file_content[6:-1]
  for i in range(len(data)):
    data[i] = data[i].replace("\n", "").lstrip()
    data[i] = list(map(float, ' '.join(data[i].split()).split(" ")))[1:]
  data = np.array(data)
  return data
  
def get_fitness(X, cities): #Ricky: X=population裡面放要經過的city順序 cities裡面放每一個city的位置
  results = np.zeros(len(X)) #Ricky: 把結果放在這邊，預設先放0
  for i in range(len(X)): #Ricky: 由第0個到len(X)依序
    distance = 0 #Ricky: 計算每一條基因的路線長度
    for j in range(len(X[i])-1): #Ricky: 由第0個到len(X)-1依序
      distance += ((cities[X[i][j]][0] - cities[X[i][j+1]][0]) ** 2  #Ricky: 計算兩點間的距離 前一點與目前這一點
        + (cities[X[i][j]][1] - cities[X[i][j+1]][1]) ** 2) **0.5
    distance += ((cities[X[i][-1]][0] - cities[X[i][0]][0]) ** 2  #Ricky: 把路線接起來 計算尾與頭的距離
      + (cities[X[i][-1]][1] - cities[X[i][0]][1]) ** 2) **0.5
    results[i] = distance #Ricky: 存地N條基因的長度
  return results #Ricky: 回傳城市與城市間距離

def selection(population, fitness, k=3): #Ricky: 目前的旅行路線 以及 目前路線中各城市間的距離
  parents = np.empty((population.shape[0], population.shape[1]))  #Ricky: 創造一個父代，大小就是目前有幾條基因(路線)，然後基因裡面有多少數字(城市)
  for parent_num in range(population.shape[0]): #Ricky: range(population.shape[0]) >> 從0開始到population.shape[0](幾條基因)
    selection_ix = np.random.randint(len(population))  #Ricky: 在隨機路線中，隨機選一個基因代號放在selection_ix中
    for ix in np.random.randint(0, len(population), k-1): #Ricky: 隨機選擇k-1個基因 
      if fitness[ix] < fitness[selection_ix]: #Ricky: 如果依序第ix條基因的路線總長度<隨機選擇的基因路線總長度
        selection_ix = ix #Ricky: 再跟下一個隨機選的k-1個基因比較
    parents[parent_num, :] = population[selection_ix, :] #Ricky: 好的基因留下
  return parents.astype('int') #Ricky: 回傳好的個體

def crossover(parents, pc=0.8):
  dim_size = parents.shape[1]
  offspring = copy.copy(parents)
  for i in range(int(parents.shape[0] / 2)):
    if np.random.random() > pc:
      x, y = 2*i, 2*i+1
      parent_1 = parents[x]
      parent_2 = parents[y]
      crossover_point = np.random.choice(dim_size, 2, replace=False)
      if crossover_point[0] < crossover_point[1]:
        p1, p2 = crossover_point[0], crossover_point[1]
      else:
        p1, p2 = crossover_point[1], crossover_point[0]
      seq_1 = parent_1[p1:p2]
      seq_2 = parent_2[p1:p2]
      pick_1, pick_2 = [], []
      for j in range(p1, dim_size):
        if parent_1[j] not in seq_2:
          pick_2.append(parent_1[j])
        if parent_2[j] not in seq_1:
          pick_1.append(parent_2[j])
      for j in range(p1):
        if parent_1[j] not in seq_2:
          pick_2.append(parent_1[j])
        if parent_2[j] not in seq_1:
          pick_1.append(parent_2[j])
      offspring_1 = pick_1
      offspring_2 = pick_2
      for j in range(p2-p1):
        offspring_1.insert(p1+j, seq_1[j])
        offspring_2.insert(p1+j, seq_2[j])

      offspring[x] = offspring_1
      offspring[y] = offspring_2
  return offspring

def mutation(offspring, pm=0.05):
  d_list = np.array([i for i in range(offspring.shape[1])])
  for i in range(offspring.shape[0]):
    if np.random.random() < pm:
      mutation = np.random.choice(dimension_size, 2, replace=False)
      temp = offspring[i][mutation[0]]
      offspring[i][mutation[0]] = offspring[i][mutation[1]]
      offspring[i][mutation[1]] = temp
  return offspring

start_time = time.time()
np.random.seed(7)

datasets = ["dantzig42.tsp", "eil101.tsp", "gr666.tsp", "pr1002.tsp"]
#datasets = ["pr1002.tsp"]
result = []


for data_name in datasets:#Ricky: 用for去輪巡每一個DATA
  cities = load_dataset(data_name)#Ricky: 取第N個DATA SET
  dimension_size = len(cities)#Ricky: 取DATA SET的個數 維度大小用cities的數量去定義 (必要len(cities)：因為要定義有多少個city)
  population_size = dimension_size//100+2 #Ricky: 有幾條基因，每一條裡面有放經過城市的順序
  iteration_size = dimension_size*5 #Ricky:需要繁衍幾代
  pc = 0.8
  pm = 0.05
  best_solution, best_fitness = [], []

  city_no = [i for i in range(dimension_size)] #Ricky: 看有多少個城市，給他做計算的編號
  population = np.array([np.random.choice(city_no, dimension_size,  #Ricky: 去隨機取city的編號，然後放入計算的編號
        replace=False) for i in range(population_size)]) #Ricky: 每條基因裡面的city不能重複且每一個城市都要取得
  fitness = np.zeros(population_size) #Ricky: return 有population_size個大小的list 裡面都放 0
  fitness = get_fitness(population, cities)  #Ricky: 去取得各城市間的距離 population裡面放要經過的city順序 cities裡面放每一個city的位置
  for iter in range(iteration_size): #Ricky: 開始繁衍
    parents = selection(population, fitness)  #Ricky: 去... 給目前的旅行路線 以及 目前路線中各城市間的距離
    offspring_crossover = crossover(parents, pc)
    offspring_mutation = mutation(offspring_crossover, pm)
    population[:] = offspring_mutation[:]
    fitness = get_fitness(offspring_crossover, cities)
  
  result.append(np.max(fitness))
  
print(result)
print(np.mean(result))
end_time = time.time()
print("Spend %0.4f sec(s)" % (end_time - start_time))

[2713.4844082566638, 2599.9951223072835, 28053.19574256985, 2841788.1688979245]
718788.7110427646
Spend 278.8782 sec(s)
